In [ ]:
import re
import sys
import time
import requests
import pprint

import pandas as pd

from bs4 import BeautifulSoup
from datetime import datetime

In [ ]:
HOME = 'https://rospotrebnadzor.ru'
START_NUM = 1
END_NUM = 70 
# Первое упоминание о коронавирусе: "О случаях заболевания пневмонией в Китайской Народной Республике" (31 декабря 2019 г.)
# На 12.11.2020 оно на 70-й странице

In [ ]:
def get_article(article_url):
    try:
        article = requests.get(article_url)
        article_soup = BeautifulSoup(article.text)
        news_title = article_soup.find('h1')
        news_detail = article_soup.find_all('div', class_='news-detail')
        news_date = article_soup.find_all('p', class_='date')
        output = [news_title.text, news_date[0].text, news_detail[0].text]
    except:
        output = ['SOMETHING_WRONG_WITH_ARTICLE', sys.exc_info()]
    return output

In [ ]:
#pprint.pprint(all_blocks)

In [ ]:
start = time.time()

all_blocks = {}
for i in range(START_NUM, END_NUM+1):
    try:
        time.sleep(0.300)
        articles_block = {}
        page_url = HOME + '/about/info/news/?PAGEN_1=' + str(i)
        page = requests.get(page_url)
        page_soup = BeautifulSoup(page.text, 'lxml')
        all_urls = [HOME + href['href'] for href in page_soup.find_all(href=re.compile('news_details'))]
        for article_url in all_urls:
            element_id = article_url.split("ELEMENT_ID=")[1]
            articles_block[element_id] = get_article(article_url)
        all_blocks[str(i)] = articles_block
        print("Block " + str(i) + "\tDone")
    except:
        all_blocks[str(i)] = ['SOMETHING_WROND_WITH_BLOCK', sys.exc_info()]
        print("Block " + str(i) + "\tError")
        
total_time = time.time()-start
print("Total time " + str(total_time))

In [ ]:
#import json
#with open('all_blocks.json', 'w') as outfile:
#    json.dump(all_blocks, outfile)

In [ ]:
#import pickle
#with open('all_blocks.pickle', 'wb') as f:
#    pickle.dump(all_blocks, f)

In [ ]:
all_articles = {}
for block in all_blocks.keys():
    all_articles.update(all_blocks[block])

In [ ]:
#import pickle
#with open('all_articles.pickle', 'wb') as f:
#    pickle.dump(all_articles, f)

In [ ]:
#import json
#with open('all_articles.json', 'w') as outfile:
#    json.dump(all_articles, outfile)

In [ ]:
all_titles = [a[0] for a in all_articles.values()]
all_titles

In [ ]:
corona_titles = [title for title in all_titles if re.findall(r"коронавирус", title)]
len(corona_titles)

In [ ]:
corona_num_articles = {}
for element_id in all_articles.keys():
    title = all_articles[element_id][0]
    if re.findall(r"COVID-2019", title):
        corona_num_articles[element_id] = all_articles[element_id]
len(corona_num_articles)

In [ ]:
corona_num_articles

In [ ]:
corona_num_dates = []
for element_id in corona_num_articles.keys():
    article_date = corona_num_articles[element_id][1]
    article_date = re.sub(r"[^0-9|\.]", "", article_date)
    article_date = re.sub(r"\.$", "", article_date)
    article_date = datetime.strptime(article_date, '%d.%m.%Y').strftime("%Y-%m-%d")
    corona_num_dates.append(article_date)
corona_num_dates    

In [ ]:
# Ищем потенциальные пробелы в сообщениях. Пока они есть в начале марта и почему-то 19-го августа 2020. 
# На сайте Роспотребнадзора данных по РФ за эти периоды нет
gaps = []
for i in range(1, len(corona_num_dates)):
    date_1 = corona_num_dates[i-1]
    date_2 = corona_num_dates[i]
    datediff = datetime.strptime(date_1, '%d.%m.%Y') - datetime.strptime(date_2, '%d.%m.%Y')
    
    if datediff.days != 1:
        gaps.append((date_1, date_2))
gaps

In [ ]:
# Обработаем начерно, просто удаляя ненужные части строк по паттернам
corona_nums = []
for element_id in corona_num_articles.keys():
    corona_text = corona_num_articles[element_id][2]
    corona_text = re.sub(r"\n\n", r"\n", corona_text)
    corona_first_sent = corona_text.split('\n')[2]
    corona_num = re.sub(r"[\xa0]+", "", corona_first_sent)
    corona_num = re.sub(r"За последние сутки в России ", "", corona_num)
    corona_num = re.sub(r"зарегистрировано подтвержденных случаев коронавирусной инфекции COVID-19 – ", "", corona_num)
    corona_num = re.sub(r"в \d\d регион", "", corona_num)
    corona_num = re.sub(r"в \d регион", "", corona_num)
    corona_num = re.sub(r"COVID-19", "", corona_num)
    corona_num = re.sub(r"COVID-2019", "", corona_num)
    corona_num = corona_num.split("Российской Федерации")[0]
    corona_num = re.sub(r"5 марта подтвержден завозной из Италии случай", "1", corona_num)
    corona_num = re.sub(r"2 марта подтвержден завозной из Италии случай", "1", corona_num)
    corona_num = re.sub(r"[^0-9]", "", corona_num)
    corona_nums.append(corona_num)
#corona_nums

In [ ]:
corona_df = pd.DataFrame(list(zip(corona_num_dates, corona_nums)),
                         columns =['Date', 'ConfirmedCases'])
corona_df.to_csv('CoronaDF.tsv', sep='\t', index=False)

In [ ]:
tests_num_articles = {}
for element_id in all_articles.keys():
    title = all_articles[element_id][0]
    if re.findall(r"бюллетень", title):
        tests_num_articles[element_id] = all_articles[element_id]
len(tests_num_articles)

In [ ]:
# ! Даты идут со сдвигом на день. 5-го октября выкладывают даты от тестах за 4-е. 

tests_num_dates = []
for element_id in tests_num_articles.keys():
    article_date = tests_num_articles[element_id][1]
    article_date = re.sub(r"[^0-9|\.]", "", article_date)
    article_date = re.sub(r"\.$", "", article_date)
    article_date = datetime.strptime(article_date, '%d.%m.%Y').strftime("%Y-%m-%d")
    tests_num_dates.append(article_date)
len(tests_num_dates) 

In [ ]:
s = "проведено 66 710 463 лабораторных исследования"
re.search(r'провед(.*?)исслед', s).group()

In [ ]:
# просто ищем по паттерну
tests_nums = []
for element_id in tests_num_articles.keys():
    tests_num = tests_num_articles[element_id][2]
    tests_num = [x.group() for x in re.finditer( r'провед(.*?)исслед', tests_num)]
    if len(tests_num) == 1:
        tests_num = tests_num[0]
    elif tests_num:
        tests_num = tests_num[1]
    else:
        tests_num = ''
    tests_num = re.sub(r"[^0-9]", "", tests_num)
    tests_nums.append(tests_num)
tests_nums

In [ ]:
# Даты идут со сдвигом на день. 5-го октября выкладывают даты от тестах за 4-е. 
corona_tests_df = pd.DataFrame(list(zip(tests_num_dates, tests_nums)),
                         columns =['Date', 'TestsPerformed'])
corona_tests_df.to_csv('CoronaTestsDF.tsv', sep='\t', index=False)